**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

In [1]:
go get -v github.com/aquasecurity/kube-bench

go: downloading github.com/aquasecurity/kube-bench v0.6.5
go: downloading github.com/aws/aws-sdk-go v1.40.51
go: downloading github.com/fatih/color v1.13.0
go: downloading github.com/golang/glog v0.0.0-20160126235308-23def4e6c14b
go: downloading github.com/spf13/cobra v1.2.1
go: downloading github.com/spf13/viper v1.9.0
go: downloading gorm.io/driver/postgres v1.1.1
go: downloading gorm.io/gorm v1.21.15
go: downloading github.com/onsi/ginkgo v1.16.4
go: downloading gopkg.in/yaml.v2 v2.4.0
go: downloading k8s.io/client-go v0.22.2
go: downloading github.com/pkg/errors v0.9.1
go: downloading github.com/mattn/go-colorable v0.1.9
go: downloading github.com/mattn/go-isatty v0.0.14
go: downloading github.com/inconshreveable/mousetrap v1.0.0
go: downloading github.com/spf13/pflag v1.0.5
go: downloading github.com/fsnotify/fsnotify v1.5.1
go: downloading github.com/magiconair/properties v1.8.5
go: downloading github.com/mitchellh/mapstructure v1.4.2
go: downloading github.com/spf13/afero v1.6.0

In [28]:
kube-bench

[INFO] 1 Master Node Security Configuration
[INFO] 1.1 Master Node Configuration Files
[PASS] 1.1.1 Ensure that the API server pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.2 Ensure that the API server pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.3 Ensure that the controller manager pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.4 Ensure that the controller manager pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.5 Ensure that the scheduler pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.6 Ensure that the scheduler pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.7 Ensure that the etcd pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.8 Ensure that the etcd pod specification file ownership is set to root:root (Automated)
[WA

In [14]:
ls

ack-1.0  cis-1.20  cis-1.6      eks-1.0.1  rh-0.7
aks-1.0  cis-1.5   config.yaml  gke-1.0    rh-1.0


In [17]:
cd ..

In [18]:
ls

CONTRIBUTING.md  cfg            go.mod       job-ack.yaml       job-master.yaml
Dockerfile       check          go.sum       job-aks.yaml       job-node.yaml
LICENSE          cmd            hack         job-eks-asff.yaml  job.yaml
NOTICE           codecov.yml    hooks        job-eks.yaml       main.go
OWNERS           docs           integration  job-gke.yaml       makefile
README.md        entrypoint.sh  internal     job-iks.yaml       mkdocs.yml


In [3]:
git clone https://github.com/aquasecurity/kube-bench

fatal: destination path 'kube-bench' already exists and is not an empty directory.


: 128

In [4]:
cd kube-bench

In [20]:
kubectl apply -f job.yaml

job.batch/kube-bench created


In [25]:
kubectl get pods

NAME                        READY   STATUS      RESTARTS   AGE
kube-bench--1-vl2h9         0/1     Completed   0          63s
my-nginx-5b56ccd65f-5tbgs   1/1     Running     0          20h
my-nginx-5b56ccd65f-7br4l   1/1     Running     0          22h
my-nginx-5b56ccd65f-lrbr4   1/1     Running     0          22h
my-nginx-5b56ccd65f-nxbdq   1/1     Running     0          22h
my-nginx-5b56ccd65f-p5k7h   1/1     Running     0          22h


In [26]:
kubectl logs kube-bench--1-vl2h9

[INFO] 1 Master Node Security Configuration
[INFO] 1.1 Master Node Configuration Files
[PASS] 1.1.1 Ensure that the API server pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.2 Ensure that the API server pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.3 Ensure that the controller manager pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.4 Ensure that the controller manager pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.5 Ensure that the scheduler pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.6 Ensure that the scheduler pod specification file ownership is set to root:root (Automated)
[PASS] 1.1.7 Ensure that the etcd pod specification file permissions are set to 644 or more restrictive (Automated)
[PASS] 1.1.8 Ensure that the etcd pod specification file ownership is set to root:root (Automated)
[WA

In [ ]:
kube-bench

In [6]:
kube-bench --help

This tool runs the CIS Kubernetes Benchmark (https://www.cisecurity.org/benchmark/kubernetes/)

Usage:
  kube-bench [flags]
  kube-bench [command]

Available Commands:
  completion  generate the autocompletion script for the specified shell
  help        Help about any command
  run         Run tests
  version     Shows the version of kube-bench.

Flags:
      --alsologtostderr                  log to standard error as well as files
      --asff                             Send the results to AWS Security Hub
      --benchmark string                 Manually specify CIS benchmark version. It would be an error to specify both --version and --benchmark flags
  -c, --check string                     A comma-delimited list of checks to run as specified in CIS document. Example --check="1.1.1,1.1.2"
      --config string                    config file (default is ./cfg/config.yaml)
  -D, --config-dir string                config directory (default "./cfg/")
      --exit-code int            

In [ ]:
ls

In [7]:
ls cfg

ack-1.0  cis-1.20  cis-1.6      eks-1.0.1  rh-0.7
aks-1.0  cis-1.5   config.yaml  gke-1.0    rh-1.0


In [8]:
cat cfg/config.yaml

---
## Controls Files.
# These are YAML files that hold all the details for running checks.
#
## Uncomment to use different control file paths.
# masterControls: ./cfg/master.yaml
# nodeControls: ./cfg/node.yaml

master:
  components:
    - apiserver
    - scheduler
    - controllermanager
    - etcd
    - flanneld
    # kubernetes is a component to cover the config file /etc/kubernetes/config that is referred to in the benchmark
    - kubernetes
    - kubelet

  kubernetes:
    defaultconf: /etc/kubernetes/config

  apiserver:
    bins:
      - "kube-apiserver"
      - "hyperkube apiserver"
      - "hyperkube kube-apiserver"
      - "apiserver"
      - "openshift start master api"
      - "hypershift openshift-kube-apiserver"
    confs:
      - /etc/kubernetes/manifests/kube-apiserver.yaml
      - /etc/kubernetes/manifests/kube-apiserver.yml
      - /etc/kubernetes/manifests/kube-apiserver.manifest
      - /var/snap/kube-apiserver/current/args
      - /var/snap/microk8s/current/args/k

In [12]:
kube-bench run -c cfg/cis-1.20

[INFO] 1 Master Node Security Configuration

== Summary master ==
0 checks PASS
0 checks FAIL
0 checks WARN
0 checks INFO

[INFO] 2 Etcd Node Configuration

== Summary etcd ==
0 checks PASS
0 checks FAIL
0 checks WARN
0 checks INFO

[INFO] 3 Control Plane Configuration

== Summary controlplane ==
0 checks PASS
0 checks FAIL
0 checks WARN
0 checks INFO

[INFO] 4 Worker Node Security Configuration

== Summary node ==
0 checks PASS
0 checks FAIL
0 checks WARN
0 checks INFO

[INFO] 5 Kubernetes Policies

== Summary policies ==
0 checks PASS
0 checks FAIL
0 checks WARN
0 checks INFO

== Summary total ==
0 checks PASS
0 checks FAIL
0 checks WARN
0 checks INFO



In [ ]:
cat job-aks.yaml

In [ ]:
cd 

In [ ]:
git clone https://github.com/kubernetes/kube-state-metrics

In [ ]:
cd kube-state-metrics

In [ ]:
make

In [ ]:
ls

In [ ]:
go build

In [ ]:
ls

In [ ]:
./kube-state-metrics --help

In [ ]:
kubectl get pods -A

In [ ]:
cd /root/go/src/github.com/kubernetes/kube-state-metrics

In [ ]:
ls

In [ ]:
make

In [ ]:
go install

In [ ]:
ls

In [ ]:
kube-state-metrics

In [ ]:
cat ~/.kube/config

In [ ]:
kube-state-metrics --kubeconfig /root/.kube/config

In [ ]:
kube-state-metrics --help

In [ ]:
curl minikube:8081

# Armosec Kubescape

Kubescape is a tool for testing Kubernetes security posture based on NSA \ MITRE ATT&CK® specifications.


https://github.com/armosec/kubescape

__Warning: next command installs software from an unknown source__

In [29]:
curl -s https://raw.githubusercontent.com/armosec/kubescape/master/install.sh | /bin/bash

Installing Kubescape...

######################################################################## 100.0%##O=#  #                                                                      

Finished Installation.

Your current version is: v1.0.135

Usage: $ kubescape scan --submit



In [30]:
kubescape scan --submit

ARMO security scanner starting
[progress] Downloading/Loading policy definitions
[success] Downloaded/Loaded policy
[progress] Accessing Kubernetes objects
W1215 10:09:17.810269  324958 warnings.go:70] policy/v1beta1 PodSecurityPolicy is deprecated in v1.21+, unavailable in v1.25+
W1215 10:09:17.819129  324958 warnings.go:70] batch/v1beta1 CronJob is deprecated in v1.21+, unavailable in v1.25+; use batch/v1 CronJob
W1215 10:09:17.867390  324958 warnings.go:70] batch/v1beta1 CronJob is deprecated in v1.21+, unavailable in v1.25+; use batch/v1 CronJob
[success] Accessed successfully to Kubernetes objects
[success] Let’s start!!!
[progress] Scanning cluster 
◓ [success] Done scanning cluster 
[control: Access Kubernetes dashboard] passed 👍
Description: Attackers who gain access to the dashboard service account or have its RBAC permissions can use its network access to retrieve information about resources in the cluster or change them. This control checks if a subject that is not dashboard

In [31]:
kubescape --help

Kubescape is a tool for testing Kubernetes security posture based on NSA \ MITRE ATT&CK® specifications.

Usage:
  kubescape [command]

Available Commands:
  completion  generate the autocompletion script for the specified shell
  config      Set configuration
  download    Download framework/control
  help        Help about any command
  scan        Scan the current running cluster or yaml files
  submit      Submit an object to the Kubescape SaaS version
  version     Get current version

Flags:
      --account string   Armo portal account ID. Default will load account ID from configMap or config file
  -h, --help             help for kubescape

Use "kubescape [command] --help" for more information about a command.
